# COMPARAISON DNN - LSTM 

Notebook généré par ChatGPT à partir du prompt :

`Quel est l'avantage des LSTM plutot qu'un simple DNN pour classifier des séquences ?.`

https://chatgpt.com/share/68021a99-a804-8012-92d7-6d621f1642a0


# 🧠 Comparaison entre DNN et LSTM pour la classification de séquences

Ce notebook montre pourquoi un **LSTM** est souvent **plus adapté qu’un réseau dense (DNN)** pour des tâches de **classification de séquences**, surtout quand l’ordre des éléments est important.

## 🎯 Objectif :
Créer deux modèles :
- Un **DNN** (réseau entièrement connecté) qui ne tient pas compte de l’ordre.
- Un **LSTM** qui prend en compte la structure séquentielle.

On comparera leurs performances sur deux tâches :
1. Classification basée sur la somme des éléments (pas de dépendance à la position).
2. Classification basée sur des critères **positionnels**.


In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np


## 📦 Définition du modèle DNN (entrée aplatie)

In [3]:

class DNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # x: [batch_size, 5]
        return self.fc2(x)


## 🔁 Définition du modèle LSTM (entrée séquentielle)

In [4]:

class LSTMClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=16, batch_first=True)
        self.fc = nn.Linear(16, 2)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn: [1, batch_size, hidden_size]
        return self.fc(hn.squeeze(0))  # [batch_size, 2]


## 🧪 Génération de données (tâche simple - somme des éléments)

In [5]:

def generate_batch(batch_size=32):
    data = []
    labels = []
    for _ in range(batch_size):
        seq = torch.rand(5)
        label = 1 if seq.sum() > 2 else 0
        data.append(seq)
        labels.append(label)
    return torch.stack(data), torch.tensor(labels)


## 🔄 Génération de données (tâche positionnelle)

In [6]:

def generate_positional_batch(batch_size=32):
    data = []
    labels = []
    for _ in range(batch_size):
        seq = torch.rand(5)
        condition = (seq[0] > 0.8) and (seq[-1] + seq[-2] < 0.5)
        label = 1 if condition else 0
        data.append(seq)
        labels.append(label)
    return torch.stack(data), torch.tensor(labels)


## 🏋️‍♂️ Fonction d'entraînement

In [7]:

def train(model, is_lstm=False, use_positional_data=False, epochs=30):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        if use_positional_data:
            X, y = generate_positional_batch()
        else:
            X, y = generate_batch()

        if is_lstm:
            X = X.unsqueeze(2)  # [batch, seq_len, features]

        outputs = model(X)
        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (outputs.argmax(1) == y).float().mean().item()
        print(f"Epoch {epoch+1:02d} - Loss: {loss.item():.4f} - Acc: {acc*100:.1f}%")


## 🚀 Testons sur les deux tâches !

### 🧪 Tâche 1 : somme des éléments — avec DNN

In [8]:

dnn = DNNClassifier()
train(dnn, is_lstm=False, use_positional_data=False)


Epoch 01 - Loss: 0.7257 - Acc: 21.9%
Epoch 02 - Loss: 0.6716 - Acc: 78.1%
Epoch 03 - Loss: 0.6254 - Acc: 75.0%
Epoch 04 - Loss: 0.6126 - Acc: 68.8%
Epoch 05 - Loss: 0.5607 - Acc: 71.9%
Epoch 06 - Loss: 0.5914 - Acc: 62.5%
Epoch 07 - Loss: 0.3303 - Acc: 96.9%
Epoch 08 - Loss: 0.3599 - Acc: 87.5%
Epoch 09 - Loss: 0.3783 - Acc: 84.4%
Epoch 10 - Loss: 0.6570 - Acc: 62.5%
Epoch 11 - Loss: 0.4452 - Acc: 78.1%
Epoch 12 - Loss: 0.2683 - Acc: 90.6%
Epoch 13 - Loss: 0.4258 - Acc: 81.2%
Epoch 14 - Loss: 0.5497 - Acc: 71.9%
Epoch 15 - Loss: 0.4045 - Acc: 81.2%
Epoch 16 - Loss: 0.4592 - Acc: 78.1%
Epoch 17 - Loss: 0.5258 - Acc: 75.0%
Epoch 18 - Loss: 0.4028 - Acc: 81.2%
Epoch 19 - Loss: 0.4519 - Acc: 78.1%
Epoch 20 - Loss: 0.3337 - Acc: 84.4%
Epoch 21 - Loss: 0.5388 - Acc: 71.9%
Epoch 22 - Loss: 0.4399 - Acc: 78.1%
Epoch 23 - Loss: 0.2806 - Acc: 87.5%
Epoch 24 - Loss: 0.2934 - Acc: 87.5%
Epoch 25 - Loss: 0.4167 - Acc: 78.1%
Epoch 26 - Loss: 0.5127 - Acc: 71.9%
Epoch 27 - Loss: 0.4266 - Acc: 75.0%
E

### 🧪 Tâche 1 : somme des éléments — avec LSTM

In [9]:

lstm = LSTMClassifier()
train(lstm, is_lstm=True, use_positional_data=False)


Epoch 01 - Loss: 0.6394 - Acc: 71.9%
Epoch 02 - Loss: 0.6003 - Acc: 78.1%
Epoch 03 - Loss: 0.6277 - Acc: 68.8%
Epoch 04 - Loss: 0.5781 - Acc: 75.0%
Epoch 05 - Loss: 0.4881 - Acc: 84.4%
Epoch 06 - Loss: 0.5873 - Acc: 71.9%
Epoch 07 - Loss: 0.5130 - Acc: 78.1%
Epoch 08 - Loss: 0.4281 - Acc: 84.4%
Epoch 09 - Loss: 0.4169 - Acc: 84.4%
Epoch 10 - Loss: 0.6642 - Acc: 71.9%
Epoch 11 - Loss: 0.7102 - Acc: 68.8%
Epoch 12 - Loss: 0.4671 - Acc: 81.2%
Epoch 13 - Loss: 0.5538 - Acc: 75.0%
Epoch 14 - Loss: 0.5931 - Acc: 71.9%
Epoch 15 - Loss: 0.5827 - Acc: 71.9%
Epoch 16 - Loss: 0.4334 - Acc: 84.4%
Epoch 17 - Loss: 0.5759 - Acc: 71.9%
Epoch 18 - Loss: 0.5322 - Acc: 75.0%
Epoch 19 - Loss: 0.4878 - Acc: 81.2%
Epoch 20 - Loss: 0.5877 - Acc: 68.8%
Epoch 21 - Loss: 0.5338 - Acc: 75.0%
Epoch 22 - Loss: 0.5075 - Acc: 78.1%
Epoch 23 - Loss: 0.4867 - Acc: 81.2%
Epoch 24 - Loss: 0.4948 - Acc: 78.1%
Epoch 25 - Loss: 0.4776 - Acc: 78.1%
Epoch 26 - Loss: 0.3794 - Acc: 87.5%
Epoch 27 - Loss: 0.3605 - Acc: 87.5%
E

### 🧪 Tâche 2 : dépendance à la position — avec DNN

In [10]:

dnn = DNNClassifier()
train(dnn, is_lstm=False, use_positional_data=True)


Epoch 01 - Loss: 0.6156 - Acc: 100.0%
Epoch 02 - Loss: 0.5265 - Acc: 100.0%
Epoch 03 - Loss: 0.4454 - Acc: 100.0%
Epoch 04 - Loss: 0.3879 - Acc: 96.9%
Epoch 05 - Loss: 0.3269 - Acc: 96.9%
Epoch 06 - Loss: 0.2392 - Acc: 100.0%
Epoch 07 - Loss: 0.2380 - Acc: 96.9%
Epoch 08 - Loss: 0.2072 - Acc: 96.9%
Epoch 09 - Loss: 0.1050 - Acc: 100.0%
Epoch 10 - Loss: 0.1568 - Acc: 96.9%
Epoch 11 - Loss: 0.0687 - Acc: 100.0%
Epoch 12 - Loss: 0.1275 - Acc: 96.9%
Epoch 13 - Loss: 0.0292 - Acc: 100.0%
Epoch 14 - Loss: 0.2523 - Acc: 93.8%
Epoch 15 - Loss: 0.1282 - Acc: 96.9%
Epoch 16 - Loss: 0.1308 - Acc: 96.9%
Epoch 17 - Loss: 0.5318 - Acc: 87.5%
Epoch 18 - Loss: 0.0102 - Acc: 100.0%
Epoch 19 - Loss: 0.0139 - Acc: 100.0%
Epoch 20 - Loss: 0.0086 - Acc: 100.0%
Epoch 21 - Loss: 0.0108 - Acc: 100.0%
Epoch 22 - Loss: 0.1483 - Acc: 96.9%
Epoch 23 - Loss: 0.1459 - Acc: 96.9%
Epoch 24 - Loss: 0.1491 - Acc: 96.9%
Epoch 25 - Loss: 0.0077 - Acc: 100.0%
Epoch 26 - Loss: 0.1427 - Acc: 96.9%
Epoch 27 - Loss: 0.1306 - 

### 🧪 Tâche 2 : dépendance à la position — avec LSTM

In [11]:

lstm = LSTMClassifier()
train(lstm, is_lstm=True, use_positional_data=True)


Epoch 01 - Loss: 0.7287 - Acc: 3.1%
Epoch 02 - Loss: 0.6830 - Acc: 93.8%
Epoch 03 - Loss: 0.6387 - Acc: 93.8%
Epoch 04 - Loss: 0.5939 - Acc: 93.8%
Epoch 05 - Loss: 0.5209 - Acc: 100.0%
Epoch 06 - Loss: 0.4763 - Acc: 96.9%
Epoch 07 - Loss: 0.3981 - Acc: 100.0%
Epoch 08 - Loss: 0.3336 - Acc: 100.0%
Epoch 09 - Loss: 0.3032 - Acc: 96.9%
Epoch 10 - Loss: 0.2507 - Acc: 96.9%
Epoch 11 - Loss: 0.1563 - Acc: 100.0%
Epoch 12 - Loss: 0.1128 - Acc: 100.0%
Epoch 13 - Loss: 0.0757 - Acc: 100.0%
Epoch 14 - Loss: 0.0521 - Acc: 100.0%
Epoch 15 - Loss: 0.1369 - Acc: 96.9%
Epoch 16 - Loss: 0.1393 - Acc: 96.9%
Epoch 17 - Loss: 0.1412 - Acc: 96.9%
Epoch 18 - Loss: 0.0129 - Acc: 100.0%
Epoch 19 - Loss: 0.1541 - Acc: 96.9%
Epoch 20 - Loss: 0.1593 - Acc: 96.9%
Epoch 21 - Loss: 0.4723 - Acc: 90.6%
Epoch 22 - Loss: 0.1637 - Acc: 96.9%
Epoch 23 - Loss: 0.3227 - Acc: 93.8%
Epoch 24 - Loss: 0.0064 - Acc: 100.0%
Epoch 25 - Loss: 0.0067 - Acc: 100.0%
Epoch 26 - Loss: 0.0065 - Acc: 100.0%
Epoch 27 - Loss: 0.0066 - Ac

# Evaluons

In [15]:
def evaluate(model, is_lstm=False):
    model.eval()
    with torch.no_grad():
        X_test, y_test = generate_positional_batch()
        if is_lstm:
            X_test = X_test.unsqueeze(2)

        outputs = model(X_test)
        acc = (outputs.argmax(1) == y_test).float().mean().item()
        print(f"✅ Test Accuracy: {acc * 100:.2f}%")
    model.train()


In [32]:
evaluate(dnn, is_lstm=False)

✅ Test Accuracy: 96.88%


In [33]:
evaluate(lstm, is_lstm=True)

✅ Test Accuracy: 100.00%


# Conclusion
Le modèle LSTM est censé mieux fonctionner pour classer les séquence selon un critère dans lequel l'ordre est important mais ce n'est pas visible dans cet exemple. 